In [69]:
# python code:
# file openscad1.py needs to be placed in the same folder (attached here)

from openscad1 import *

# variation in path extrude

# profile of the fan blade

p0=cr(pts1([[0,0,.5],[20,-2,300],[20,-5,100],[5,5,50],[-5,5,100],[-20,0,300],[-20,-2,.5]]),20)

# blade profile smoothened with bezier
sec=bezier(p0,200)

# xval is the projection of fan profile on x-axis
xval=array(sec[:int(len(sec)/2)])[:,0]

# generating helical path to extrude the profile
extrude_angle=90
extrude_height=30
inner_radius=30
path=[[inner_radius*cos(d2r(extrude_angle/xval[-1]*i)),inner_radius*sin(d2r(extrude_angle/xval[-1]*i)),extrude_height/xval[-1]*i] for i in xval]
path1=[[inner_radius*cos(d2r(-10)),inner_radius*sin(d2r(-10)),0],[inner_radius*cos(d2r(-5)),inner_radius*sin(d2r(-5)),0.0],[inner_radius*cos(d2r(-5)),inner_radius*sin(d2r(-5)),0.0],[inner_radius*cos(d2r(0)),inner_radius*sin(d2r(0)),0.0]]
path=bezier(path1+path[20:],len(xval))
# linear extrude to convert fan blade profile to a 3d solid
sol=linear_extrude(sec,45)

# a little complicated code to extrude the solid along the given path after changing the orientation
sol1=translate([45,0,0],q_rot(['y-90'],sol))
sol1=c2ro(sol1,1)
sol1=c3t2(sol1)
sol2=[]
for p in sol1:
    r1=array([l_len(p1) for p1 in seg(p) ]).min()/5
    s1=cr([[p[0][0],p[0][1],r1],[p[1][0],p[1][1],r1],[p[2][0],p[2][1],r1],[p[3][0],p[3][1],r1]],10)
    sol2.append(s1)
    
p1=path[:-1]
p2=path[1:]
p1,p2=array([p1,p2])
v1=p2-p1
u1=v1/norm(v1,axis=1).reshape(-1,1)
v2=concatenate([[u1[0]],(u1[1:]+u1[:-1])/2,[u1[-1]]])
sec2=[]
for i in range(len(path)):
    sec1=translate(path[i],sec2vector(v2[i],sol2[i]))
    sec2.append(sec1)

sec2=slice_sol(sec2,10)

# file path needs to be modified
with open('/Users/sanjeevprabhakar/iCloud Drive (Archive)/Documents/Download/openscad/trial.scad','w+') as f:
    f.write(f'''

for(i=[0:360/3:359])
rotate([0,0,i])
mirror([0,1,0]){swp(sec2)}

''')
